# Задача №8
Сравниение машинных переводчиков.

In [2]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import scipy.stats as st
import seaborn as sns
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer

Откроем и считаем файлы с переводами:

In [3]:
def open_f(name):
    file = open(name, 'r')
    text = file.read().lower().split('\n')
    data = []
    tokenizer = RegexpTokenizer(r'\w+')
    for a in text[:-1]:
        data.append(tokenizer.tokenize(a))
    return data

In [6]:
langs = ["en_de_", "en_ru_", "kk_ru_"]
machines = ['n', 'p', 'z']
canon = 'gold'
path = 'mt/'
texts = []
gold = []
for i in range(0,3):
    texts.append([])
    for j in range(0,3):
        texts[i].append(open_f(path + langs[i] + machines[j] + '.txt'))
    gold.append(open_f(path + langs[i] + canon + '.txt'))
    

Теперь посчитаем BLEU скоры:

In [7]:
bleu = []
for i in range(0,3):
    bleu.append([])
    for j in range(0,3):
        bleu[i].append([])
        num = 0
        for sen in texts[i][j]:
            bleu[i][j].append(nltk.translate.bleu_score.sentence_bleu([gold[i][num]], sen))
            num += 1

/home/tim/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/tim/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/tim/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunc

In [8]:
for i in range(0,3):
    for j in range(0,3):
        print(st.shapiro(bleu[i][j]))

(0.7101461887359619, 0.0011693767737597227)
(0.7209776639938354, 0.001582909724675119)
(0.6593667268753052, 0.00028449439560063183)
(0.7364778518676758, 0.002442906377837062)
(0.8187993168830872, 0.024508699774742126)
(0.6658506393432617, 0.00034055986907333136)
(0.5918524265289307, 4.4228807382751256e-05)
(0.7261315584182739, 0.001828456181101501)
(0.640485405921936, 0.00016868000966496766)


Достаточно заметно,что распределение максимально непохоже на нормальное. Так что сравнивать средние будем выборочным критерием.

Теперь попробуем определить лучшие переводчики для каждой пары языков по среднему значению:

In [9]:
best = []
bleu_np = np.array(bleu)
for i in range(0,3):
    maxb = [0, 0]
    for j in range(0,3):
        k = bleu_np[i, j].mean()
        if maxb[0] < k:
            maxb = [k, j]
        print(i, j, k)
    best.append(maxb)

0 0 0.12334295070838248
0 1 0.11321599701471467
0 2 0.05463601263714318
1 0 0.14093320088587458
1 1 0.1794806754140275
1 2 0.10810424243774971
2 0 0.20758282761173405
2 1 0.2392390406367968
2 2 0.4


Выборочный критерий для сравнения с лидерами выявленными на предыдущем шаге:

In [10]:
from permute.core import one_sample
for i in range(0,3):
    k = best[i][1]
    for j in range(0,3):
        if k == j: continue
        print(i, j, k, one_sample(bleu_np[i][k], bleu_np[i][j], alternative='less'))

0 1 0 (0.62425, 0.010126953693667808)
0 2 0 (0.87526, 0.06870693807123929)
1 0 1 (0.81494, 0.03854747452815292)
1 2 1 (0.90611, 0.07137643297627778)
2 0 2 (0.81364, 0.19241717238826594)
2 1 2 (0.87612, 0.16076095936320317)


Итого, как ни странно, при уровне значимости 0.1 можно выявить лидеров, соответственно машина N для англо-немецкого перевода и машина Р для англо-русского. Однако при уровне значимости 0.2 можно принять и гипотезу, что Z лучший переводчик на третьей паре языков. На самом деле, это все не совсем правда, вроде, так как здесь рассматривается не та нулевая гипотеза, но так как ничего лучше не придумал и не нашел, останется этот вариант.